<a href="https://colab.research.google.com/github/steinjm21/ColabInClass/blob/main/matplotlibProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/steinjm21/matplotlibProject.git

fatal: destination path 'matplotlibProject' already exists and is not an empty directory.


Defining function to be outputted to .csv

In [22]:
import numpy as np
import sys
import datetime
import math

In [23]:
def inside_circle(total_count):
    x = np.random.uniform(size=total_count)
    y = np.random.uniform(size=total_count)
    radii = np.sqrt(x*x + y*y)
    count = len(radii[np.where(radii<=1.0)])
    return count

I found using np.empty and np.vstack to be more efficient (it actually worked) with the code I had written and tried to painstakingly edit for hours.

In [33]:
if __name__ == '__main__':
    grid = np.empty((0, 2))
    for n in range(100, 10000):
      n_samples = int(n)
      start_time = datetime.datetime.now()
      counts = inside_circle(n_samples)
      my_pi = 4.0 * counts / n_samples
      end_time = datetime.datetime.now()
      elapsed_time = (end_time - start_time).total_seconds()
      size_of_float = np.dtype(np.float64).itemsize
      memory_required = 3 * n_samples * size_of_float / (1024**3)
      abs_err = abs(my_pi - math.pi)
      est_rel_err = abs(abs_err / math.pi) / abs(math.pi)
      actual_rel_err = 0
      grid = np.vstack([grid, [est_rel_err, actual_rel_err]])
    print("# of Samples: {}, Pi: {}, memory: {} GiB, time: {} s".format(
          n_samples, my_pi, memory_required, elapsed_time))

# of Samples: 9999, Pi: 3.1567156715671567, memory: 0.00022349506616592407 GiB, time: 0.000287 s


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 19800 and the array at index 1 has size 2